In [1]:
    import numpy as np
import pandas as pd

In [1]:
from src.data import all_functions as af

In [2]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_team_player_game_dataframe(team_dict, week="next", season=2020):
    """

    @param team_dict:
    @param week:
    @param season:
    @return:
    """
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season) # Run once only
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    return filtered_games_week, player_games_dictionary

In [76]:
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af.NUNN_OF_YALL_BETTA,
                                                                            week="current",
                                                                              season=2020)

[2021-03-09 21:24:43,631 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:24:43,633 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 5700.9768097400665
[2021-03-09 21:24:43,635 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-03-09 21:24:43,635 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2021-03-09 21:24:50,107 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:24:50,117 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 6.481400012969971
[2021-03-09 21:24:50,118 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:24:52,411 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:24:52,415 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 8.778918981552124
[2021-03-09 21:24:52,417 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:24:53,394 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:24:53,398 DEBUG] [yahoo_oauth.oauth.token_is_vali

In [86]:
player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
df = player_games.loc[player_games.index.repeat(player_games[0])]
df

,0
Malcolm Brogdon,5
Malcolm Brogdon,5
Malcolm Brogdon,5
Malcolm Brogdon,5
Malcolm Brogdon,5
...,...
Pascal Siakam,5
Pascal Siakam,5
Pascal Siakam,5
Pascal Siakam,5


In [102]:
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-03-09 22:31:33,666 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 22:31:33,668 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 451.64334321022034
[2021-03-09 22:31:33,669 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [ ]:
nunn = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [ ]:
nunn

In [ ]:
nunn.drop("PLAYER", axis=1, inplace=True)
nunn.drop("mean", inplace=True)
nunn = nunn.apply(pd.to_numeric)

In [87]:
# sc = af.yahoo_fantasy_api_authentication()
# league = af.yahoo_fantasy_league(sc)
# team_9cat_averages = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)
# team_9cat_averages.drop("mean", inplace=True)
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af.NUNN_OF_YALL_BETTA,
                                                                              week="current",
                                                                              season=2020)
player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
player_games = player_games.loc[player_games.index.repeat(player_games[0])]
player_games.reset_index(inplace=True)
player_games.columns = [["PLAYER", "G"]]
# simulation_dataset = player_games.merge(team_9cat_averages, on="PLAYER")

[2021-03-09 21:27:47,155 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:27:47,157 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 183.52103352546692
[2021-03-09 21:27:47,158 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:27:52,750 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:27:52,754 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 189.11841678619385
[2021-03-09 21:27:52,755 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:27:53,745 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:27:53,748 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 190.11219310760498
[2021-03-09 21:27:53,749 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:27:55,921 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:27:55,924 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 192.2884237766266
[2021-03-09 21:27:55,925 DEBUG] [yahoo_oauth.oaut

In [84]:
# player_games.merge(team_9cat_averages, left_on="PLAYER", right_on="PLAYER")
team_9cat_averages

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
8,Cameron Johnson,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32


In [85]:
team_9cat_averages.merge(player_games, left_on="PLAYER", right_on="PLAYER")

KeyError: 'PLAYER'

In [ ]:
team_9cat_averages.to_clipboard(index=False)
team_9cat_averages=pd.read_clipboard(sep='\t')

player_games.to_clipboard(index=False)
player_games=pd.read_clipboard(sep='\t')

In [78]:
pd.merge(team_9cat_averages, player_games, on=["PLAYER"])

NameError: name 'player_games' is not defined

In [5]:
def make_simulation_dataset(team_dict, week="current"):
    """

    @param team_dict:
    @param week:
    @return:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team_9cat_averages = af.team_9cat_average_stats(team_dict, league)
    team_9cat_averages.drop("mean", inplace=True)
    filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(team_dict,
                                                                                  week=week,
                                                                                  season=2020)
    player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
    player_games = player_games.loc[player_games.index.repeat(player_games[0])]
    player_games.reset_index(inplace=True)
    player_games.columns = [["PLAYER", "G"]]
    team_9cat_averages.to_clipboard(index=False)
    team_9cat_averages=pd.read_clipboard(sep='\t')
    player_games.to_clipboard(index=False)
    player_games=pd.read_clipboard(sep='\t')
    simulation_dataset = pd.merge(team_9cat_averages, player_games, on=["PLAYER"])
    simulation_dataset.drop(["PLAYER", "G"], axis=1, inplace=True)
    simulation_dataset = simulation_dataset.apply(pd.to_numeric)
    return simulation_dataset

In [132]:
simulation_dataset = make_simulation_dataset(af.NUNN_OF_YALL_BETTA, week="current")
simulation_dataset

[2021-03-08 21:09:18,815 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:09:18,817 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1566.2080564498901
[2021-03-08 21:09:18,818 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:16,873 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:16,877 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1624.2677590847015
[2021-03-08 21:10:16,878 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:25,193 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:25,197 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1632.5879747867584
[2021-03-08 21:10:25,197 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:28,527 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:28,529 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1635.9200959205627
[2021-03-08 21:10:28,530 DEBUG] [yahoo_oauth.oau

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
2,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
3,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
4,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
...,...,...,...,...,...,...,...,...,...
67,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
68,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
69,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
70,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17


In [6]:
gaussian_stats = ['FG_PCT', 'FT_PCT']
poison_stats = ['FG3M', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']

def simulate_stats(team_df, n_reps):

    pois_stats = team_df.loc[:, poison_stats].values
    gaus_stats = team_df.loc[:, gaussian_stats].values
    pois = np.random.poisson(lam=pois_stats,
                             size=(n_reps, *pois_stats.shape))
    pois = np.maximum(0, pois)
    gaus = np.random.normal(loc=gaus_stats, scale=0.2 * gaus_stats,
                            size=(n_reps, *gaus_stats.shape))
    gaus = np.clip(gaus, 0, 1)

    aggr_pois_stats = pois.sum(axis=1)
    aggr_gaus_stats = gaus.mean(axis=1)
    aggr_stats = np.concatenate((aggr_gaus_stats, aggr_pois_stats),
                                axis=1)
    return aggr_stats

In [133]:
nunn_simulated = simulate_stats(simulation_dataset, 10000)
nunn_simulated

array([[  0.46597222,   0.82735082, 141.        , ...,  71.        ,
         29.        ,  94.        ],
       [  0.45297182,   0.81913327, 116.        , ...,  67.        ,
         22.        , 113.        ],
       [  0.46966351,   0.78848219, 152.        , ...,  66.        ,
         30.        , 142.        ],
       ...,
       [  0.46886232,   0.79490553, 106.        , ...,  61.        ,
         38.        , 129.        ],
       [  0.47487524,   0.80112521, 131.        , ...,  71.        ,
         37.        , 101.        ],
       [  0.47467781,   0.80851758, 153.        , ...,  71.        ,
         30.        , 118.        ]])

In [63]:
crabbe = af.team_9cat_average_stats(af.CRABBEHERBYTHEPUSSY, league)
crabbe.drop("PLAYER", axis=1, inplace=True)
crabbe.drop("mean", inplace=True)
crabbe = crabbe.apply(pd.to_numeric)

In [65]:
crabbe_simulated = simulate_stats(crabbe, 100000)

In [68]:
perc_win = ((nunn_simulated > crabbe_simulated).sum(axis=0)
            / 100000 * 100)
perc_win

array([26.314, 45.202, 71.874, 68.78 , 28.811, 87.807, 60.133,  4.706,
       42.451])

In [71]:
perc_win_df = pd.DataFrame(perc_win,
                           index=gaussian_stats + poison_stats,
                           columns=['WinProb'])
perc_win_df.loc['TOV'] = 100 - perc_win_df.loc['TOV']
perc_win_df


,WinProb
FG_PCT,26.314
FT_PCT,45.202
FG3M,71.874
PTS,68.780
REB,28.811
AST,87.807
STL,60.133
BLK,4.706
TOV,57.549


In [7]:
gaussian_stats = ['FG_PCT', 'FT_PCT']
poison_stats = ['FG3M', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']

def matchup_simulation(team1_dict, team2_dict, n_reps=100000, week="current"):
    """

    @param team1_dict:
    @param team2_dict:
    @param n_reps:
    @return:
    """
    team1_simulation_dataset = make_simulation_dataset(team1_dict, week=week)
    team2_simulation_dataset = make_simulation_dataset(team2_dict, week=week)
    team1_sim_stats = simulate_stats(team1_simulation_dataset, n_reps)
    team2_sim_stats = simulate_stats(team2_simulation_dataset, n_reps)
    perc_win = ((team1_sim_stats > team2_sim_stats).sum(axis=0)
                / n_reps * 100)
    perc_win_df = pd.DataFrame(perc_win,
                               index=gaussian_stats + poison_stats,
                               columns=['WinProb'])
    perc_win_df.loc['TOV'] = 100 - perc_win_df.loc['TOV']
    return perc_win_df

In [35]:
simulated_cats = matchup_simulation(af.NUNN_OF_YALL_BETTA, af.RUSTY_CUNTBROOKS, n_reps=100000,
                                    week="current")

[2021-03-09 20:14:40,711 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 20:14:40,713 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1498.0572264194489
[2021-03-09 20:14:40,714 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 20:15:43,786 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 20:15:43,788 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1561.1318709850311
[2021-03-09 20:15:43,789 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 20:15:51,139 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 20:15:51,142 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1568.4858796596527
[2021-03-09 20:15:51,143 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 20:15:52,302 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 20:15:52,307 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1569.6508402824402
[2021-03-09 20:15:52,309 DEBUG] [yahoo_oauth.oau

In [36]:
simulated_cats.T

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
WinProb,5.508,98.851,98.589,71.982,36.807,86.778,28.1,1.74,72.537


In [37]:
simulated_cats.T

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
WinProb,5.508,98.851,98.589,71.982,36.807,86.778,28.1,1.74,72.537


In [68]:
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)


[2021-03-09 20:39:26,213 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 20:39:26,215 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2983.5594432353973
[2021-03-09 20:39:26,217 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [20]:
# Single team roster
tm = league.to_team('402.l.55374.t.6')
roster = tm.roster()


In [41]:
required_fields = ["name", "status"]
player_name_list = list()
player_status_list = list()
for n in range(len(roster)):
    player_status_dict = {key:value for key, value in roster[n].items() if key in required_fields}
    player_name_list.append(player_status_dict["name"])
    player_status_list.append(player_status_dict["status"])

In [42]:
player_status_dataframe = pd.DataFrame(columns=["PLAYER", "status"])
player_status_dataframe["PLAYER"] = player_name_list
player_status_dataframe["status"] = player_status_list

In [43]:
player_status_dataframe

,PLAYER,status
0,Malcolm Brogdon,
1,Duncan Robinson,
2,Patty Mills,
3,Gary Trent Jr.,
4,Jae Crowder,
5,Jeremy Lamb,
6,Enes Kanter,
7,Nikola Jokic,
8,Cameron Johnson,INJ
9,Jimmy Butler,


In [34]:
team_9cat_averages = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,status
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33,
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06,
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73,
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91,
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76,
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17,
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17,
8,Cameron Johnson,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76,INJ
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32,


In [61]:
NAME_LOWER_COLUMN = "name"
STATUS_LOWER_COLUMN = "status"

def make_player_status_dataframe(team_dict):
    """

    @param team_dict:
    @return:
    """
    tm = league.to_team(team_dict[TEAM_KEY])
    roster = tm.roster()
    required_fields = [NAME_LOWER_COLUMN, STATUS_LOWER_COLUMN]
    player_name_list = list()
    player_status_list = list()
    player_status_dataframe = pd.DataFrame(columns=[PLAYER_COLUMN, STATUS_LOWER_COLUMN])
    for n in range(len(roster)):
        player_status_dict = {key:value for key, value in roster[n].items() if key in required_fields}
        player_name_list.append(player_status_dict[NAME_LOWER_COLUMN])
        player_status_list.append(player_status_dict[STATUS_LOWER_COLUMN])
    player_status_dataframe[PLAYER_COLUMN] = player_name_list
    player_status_dataframe[STATUS_LOWER_COLUMN] = player_status_list
    return player_status_dataframe

In [57]:
nunn_status = make_player_status_dataframe(af.NUNN_OF_YALL_BETTA)
combined_df = team_9cat_averages.merge(nunn_status, on="PLAYER")
combined_df

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,status
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33,
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06,
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73,
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91,
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76,
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17,
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17,
8,Cameron Johnson,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76,INJ
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32,


In [52]:
STATUS_TO_REMOVE = ["INJ", "O", "NA"]

In [60]:
df = combined_df[~combined_df['status'].isin(STATUS_TO_REMOVE)].drop("status", axis=1)
df

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32
10,Wendell Carter Jr.,0.535,0.733,0.30,12.26,7.78,2.26,0.70,0.83,1.87


In [62]:
STATUS_TO_REMOVE = ["INJ", "O", "NA"]
def make_filtered_status_dataframe(team_9cat_average_df, player_status_df):
    """

    @param team_9cat_average_df:
    @param player_status_df:
    @return:
    """
    merged_df = team_9cat_average_df.merge(player_status_df, on=PLAYER_COLUMN)
    filtered_status_df = merged_df[~merged_df[STATUS_LOWER_COLUMN].isin(STATUS_TO_REMOVE)].drop\
        (STATUS_LOWER_COLUMN, axis=1)
    return filtered_status_df

In [63]:
make_filtered_status_dataframe(team_9cat_averages, nunn_status)

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32
10,Wendell Carter Jr.,0.535,0.733,0.30,12.26,7.78,2.26,0.70,0.83,1.87


In [9]:
STATUS_TO_REMOVE = ["INJ", "O", "NA"]
NAME_LOWER_COLUMN = "name"
STATUS_LOWER_COLUMN = "status"
gaussian_stats = ['FG_PCT', 'FT_PCT']
poison_stats = ['FG3M', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']

import numpy as np


def make_player_status_dataframe(team_dict):
    """

    @param team_dict:
    @return:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    tm = league.to_team(team_dict[TEAM_KEY])
    roster = tm.roster()
    required_fields = [NAME_LOWER_COLUMN, STATUS_LOWER_COLUMN]
    player_name_list = list()
    player_status_list = list()
    player_status_dataframe = pd.DataFrame(columns=[PLAYER_COLUMN, STATUS_LOWER_COLUMN])
    for n in range(len(roster)):
        player_status_dict = {key:value for key, value in roster[n].items() if key in required_fields}
        player_name_list.append(player_status_dict[NAME_LOWER_COLUMN])
        player_status_list.append(player_status_dict[STATUS_LOWER_COLUMN])
    player_status_dataframe[PLAYER_COLUMN] = player_name_list
    player_status_dataframe[STATUS_LOWER_COLUMN] = player_status_list
    return player_status_dataframe


def make_filtered_status_dataframe(team_9cat_average_df, player_status_df):
    """

    @param team_9cat_average_df:
    @param player_status_df:
    @return:
    """
    merged_df = team_9cat_average_df.merge(player_status_df, on=PLAYER_COLUMN)
    filtered_status_df = merged_df[~merged_df[STATUS_LOWER_COLUMN].isin(STATUS_TO_REMOVE)].drop\
        (STATUS_LOWER_COLUMN, axis=1)
    return filtered_status_df


def make_simulation_dataset(team_dict, team_9cat_roster_dataframe=None, week="current", \
                                                                        include_injured=False):
    """

    @param team_dict:
    @param week:
    @param include_injured:
    @return:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if team_9cat_roster_dataframe is None:
        team_9cat_averages = af.team_9cat_average_stats(team_dict, league)
    else:
        team_9cat_averages = team_9cat_roster_dataframe
    team_9cat_averages.drop("mean", inplace=True)
    filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(team_dict,
                                                                                  week=week,
                                                                                  season=2020)
    player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
    player_games = player_games.loc[player_games.index.repeat(player_games[0])]
    player_games.reset_index(inplace=True)
    player_games.columns = [["PLAYER", "G"]]
    team_9cat_averages.to_clipboard(index=False)
    team_9cat_averages=pd.read_clipboard(sep='\t')
    player_games.to_clipboard(index=False)
    player_games=pd.read_clipboard(sep='\t')
    simulation_dataset = pd.merge(team_9cat_averages, player_games, on=["PLAYER"])
    if include_injured == False:
        player_status_df = make_player_status_dataframe(team_dict)
        simulation_dataset = make_filtered_status_dataframe(simulation_dataset, player_status_df)
    simulation_dataset.drop(["PLAYER", "G"], axis=1, inplace=True)
    simulation_dataset.fillna(simulation_dataset.mean(), inplace=True)
    simulation_dataset = simulation_dataset.apply(pd.to_numeric)
    return simulation_dataset


def simulate_stats(team_df, n_reps):

    pois_stats = team_df.loc[:, poison_stats].values
    gaus_stats = team_df.loc[:, gaussian_stats].values
    pois = np.random.poisson(lam=pois_stats,
                             size=(n_reps, *pois_stats.shape))
    pois = np.maximum(0, pois)
    gaus = np.random.normal(loc=gaus_stats, scale=0.2 * gaus_stats,
                            size=(n_reps, *gaus_stats.shape))
    gaus = np.clip(gaus, 0, 1)

    aggr_pois_stats = pois.sum(axis=1)
    aggr_gaus_stats = gaus.mean(axis=1)
    aggr_stats = np.concatenate((aggr_gaus_stats, aggr_pois_stats),
                                axis=1)
    return aggr_stats


def matchup_simulation(team1_dict, team2_dict, team1_9cat_roster_dataframe=None,
                       team2_9cat_roster_dataframe=None, n_reps=100000, week="current",
include_injured=False):
    """

    @param team1_dict:
    @param team2_dict:
    @param n_reps:
    @return:
    """
    team1_simulation_dataset = make_simulation_dataset(team1_dict, week=week,
                                                       include_injured=include_injured)
    team2_simulation_dataset = make_simulation_dataset(team2_dict, week=week,
                                                       include_injured=include_injured)
    team1_sim_stats = simulate_stats(team1_simulation_dataset, n_reps)
    team2_sim_stats = simulate_stats(team2_simulation_dataset, n_reps)
    perc_win = ((team1_sim_stats > team2_sim_stats).sum(axis=0)
                / n_reps * 100)
    perc_win_df = pd.DataFrame(perc_win,
                               index=gaussian_stats + poison_stats,
                               columns=['WinProb'])
    perc_win_df.loc['TOV'] = 100 - perc_win_df.loc['TOV']
    return perc_win_df.T


In [90]:
simulation_dataset = make_simulation_dataset(af.NUNN_OF_YALL_BETTA, week="current",
                                           include_injured=False)
print(simulation_dataset.shape)

[2021-03-09 21:31:14,642 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:31:14,646 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 391.0106282234192
[2021-03-09 21:31:14,647 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:32:17,429 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:32:17,432 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 453.79662561416626
[2021-03-09 21:32:17,433 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:32:24,127 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:32:24,128 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 460.4921052455902
[2021-03-09 21:32:24,129 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:32:25,085 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:32:25,088 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 461.4518632888794
[2021-03-09 21:32:25,088 DEBUG] [yahoo_oauth.oauth.

(59, 10)


In [93]:
simulated_cats = matchup_simulation(af.NUNN_OF_YALL_BETTA, af.RUSTY_CUNTBROOKS, n_reps=100000,
                                    week="current", include_injured=False)

[2021-03-09 21:36:09,718 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:36:09,720 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 686.0837898254395
[2021-03-09 21:36:09,720 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:37:06,936 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:37:06,940 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 743.3039717674255
[2021-03-09 21:37:06,940 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:37:12,582 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:37:12,586 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 748.9500706195831
[2021-03-09 21:37:12,586 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 21:37:13,602 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 21:37:13,606 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 749.970118522644
[2021-03-09 21:37:13,606 DEBUG] [yahoo_oauth.oauth.to

In [100]:
simulated_cats.T.applymap(np.int64).astype(str) + '%'

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
WinProb,18%,99%,98%,89%,71%,42%,21%,2%,66%


In [10]:
simulated_cats = matchup_simulation(af.NUNN_OF_YALL_BETTA, af.WAKANDA_FOREVER,
                                    n_reps=100000, week="next", include_injured=False)


[2021-03-15 19:49:36,820 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-15 19:49:36,822 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2763.2689344882965
[2021-03-15 19:49:36,822 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-15 19:50:37,776 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-15 19:50:37,780 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2824.2266681194305
[2021-03-15 19:50:37,782 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-15 19:50:44,334 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-15 19:50:44,338 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2830.7843539714813
[2021-03-15 19:50:44,339 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-15 19:50:45,256 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-15 19:50:45,258 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2831.7041923999786
[2021-03-15 19:50:45,259 DEBUG] [yahoo_oauth.oau

In [113]:
wak = make_simulation_dataset(af.WAKANDA_FOREVER, week="current", include_injured=False)

[2021-03-09 22:41:06,971 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 22:41:06,974 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1024.9493746757507
[2021-03-09 22:41:06,975 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 22:42:05,720 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 22:42:05,723 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1083.698678970337
[2021-03-09 22:42:05,724 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 22:42:11,395 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 22:42:11,398 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1089.3739182949066
[2021-03-09 22:42:11,399 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-09 22:42:12,424 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-09 22:42:12,429 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1090.4051764011383
[2021-03-09 22:42:12,430 DEBUG] [yahoo_oauth.oaut

In [4]:
wak.info()
def simulate_stats(team_df, n_reps):

    pois_stats = team_df.loc[:, poison_stats].values
    gaus_stats = team_df.loc[:, gaussian_stats].values
    print(pois_stats)
    pois = np.random.poisson(lam=pois_stats,
                             size=(n_reps, *pois_stats.shape))
    pois = np.maximum(0, pois)
    gaus = np.random.normal(loc=gaus_stats, scale=0.2 * gaus_stats,
                            size=(n_reps, *gaus_stats.shape))
    gaus = np.clip(gaus, 0, 1)

    aggr_pois_stats = pois.sum(axis=1)
    aggr_gaus_stats = gaus.mean(axis=1)
    aggr_stats = np.concatenate((aggr_gaus_stats, aggr_pois_stats),
                                axis=1)
    return aggr_stats

NameError: name 'wak' is not defined

In [109]:
simulate_stats(wak, 10)

[[ 2.95 18.45  3.6   4.55  1.05  0.35  2.2 ]
 [ 2.95 18.45  3.6   4.55  1.05  0.35  2.2 ]
 [ 2.95 18.45  3.6   4.55  1.05  0.35  2.2 ]
 [ 2.95 18.45  3.6   4.55  1.05  0.35  2.2 ]
 [ 2.65 21.76  4.18  4.59  1.15  0.32  2.29]
 [ 2.65 21.76  4.18  4.59  1.15  0.32  2.29]
 [ 2.65 21.76  4.18  4.59  1.15  0.32  2.29]
 [ 2.65 21.76  4.18  4.59  1.15  0.32  2.29]
 [ 2.65 21.76  4.18  4.59  1.15  0.32  2.29]
 [ 1.75  8.5   3.82  2.14  0.89  0.79  0.96]
 [ 1.75  8.5   3.82  2.14  0.89  0.79  0.96]
 [ 1.75  8.5   3.82  2.14  0.89  0.79  0.96]
 [ 1.75  8.5   3.82  2.14  0.89  0.79  0.96]
 [ 0.48  4.7   2.48  0.56  0.3   0.37  0.56]
 [ 0.48  4.7   2.48  0.56  0.3   0.37  0.56]
 [ 0.48  4.7   2.48  0.56  0.3   0.37  0.56]
 [ 0.48  4.7   2.48  0.56  0.3   0.37  0.56]
 [ 0.18 25.59  7.21  3.44  0.94  0.71  2.59]
 [ 0.18 25.59  7.21  3.44  0.94  0.71  2.59]
 [ 0.18 25.59  7.21  3.44  0.94  0.71  2.59]
 [ 0.18 25.59  7.21  3.44  0.94  0.71  2.59]
 [ 0.18 25.59  7.21  3.44  0.94  0.71  2.59]
 [ 0.18 25

ValueError: lam value too large

In [114]:
wak.head(60)

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,G
0,Kemba Walker,0.388,0.914,2.95,18.45,3.60,4.55,1.05,0.35,2.20,4
1,Kemba Walker,0.388,0.914,2.95,18.45,3.60,4.55,1.05,0.35,2.20,4
2,Kemba Walker,0.388,0.914,2.95,18.45,3.60,4.55,1.05,0.35,2.20,4
3,Kemba Walker,0.388,0.914,2.95,18.45,3.60,4.55,1.05,0.35,2.20,4
4,Jamal Murray,0.487,0.846,2.65,21.76,4.18,4.59,1.15,0.32,2.29,5
5,Jamal Murray,0.487,0.846,2.65,21.76,4.18,4.59,1.15,0.32,2.29,5
6,Jamal Murray,0.487,0.846,2.65,21.76,4.18,4.59,1.15,0.32,2.29,5
7,Jamal Murray,0.487,0.846,2.65,21.76,4.18,4.59,1.15,0.32,2.29,5
8,Jamal Murray,0.487,0.846,2.65,21.76,4.18,4.59,1.15,0.32,2.29,5
9,Patrick Beverley,0.446,0.795,1.75,8.50,3.82,2.14,0.89,0.79,0.96,4


6

In [83]:
other_team = abs(simulated_cats.subtract(100))
other_team

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
WinProb,76.282,18.665,1.206,0.0,0.001,1.825,7.14,82.119,74.49


In [84]:
total_df = simulated_cats.append(other_team)
total_df

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
WinProb,23.718,81.335,98.794,100.0,99.999,98.175,92.86,17.881,25.51
WinProb,76.282,18.665,1.206,0.0,0.001,1.825,7.14,82.119,74.49


In [85]:
total_df[""] = [af.NUNN_OF_YALL_BETTA["name"], af.RUSTY_CUNTBROOKS["name"]]
total_df.reset_index(drop=True, inplace=True)
total_df.set_index("", inplace=True)

In [86]:
count = 0
count2 = 0
for value, value2 in zip(list(simulated_cats.values[0]), list(other_team.values[0])):
    if float(value) > 50:
        count+=1
    if float(value2) > 50:
        count2+=1
print(count)
print(count2)

6
3


In [88]:

total_df['CATS'] = [count, count2]

In [89]:
total_df


,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,CATS
PLAYER,,,,,,,,,,
Nunn of Y'all Betta,23.718,81.335,98.794,100.0,99.999,98.175,92.86,17.881,25.51,6
Russty Cuntbrooks,76.282,18.665,1.206,0.0,0.001,1.825,7.14,82.119,74.49,3


In [ ]:
    simulated_cats = perc_win_df.T
    other_team = abs(simulated_cats.subtract(100))
    total_df = simulated_cats.append(other_team)
    total_df[""] = [team1_dict["name"], team2_dict["name"]]
    total_df.set_index("")
    count = 0
    count2 = 0
    for value, value2 in zip(list(simulated_cats.values[0]), list(other_team.values[0])):
        if float(value) > 50:
            count += 1
        if float(value2) > 50:
            count2 += 1
    total_df['CATS'] = [count, count2]